Project Outline 
1.   We are going to scrape imdb top 1000 from multiple pages
2.   We will get a list of imdb top 1000 sorted by imdb rating descending as of 05-08-2022




# Importing Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from time import sleep
from random import randint

# Creating List for column names

In [2]:
movie_name = []
year = []
time = []
rating = []
metascore = []
votes = []
gross = []
certificate = []
genre = []
description = []
director = []
poster = []

# Use beautiful soup to parse and extract information

In [3]:
pages = np.arange(1,1000,100)

In [4]:
for page in pages:
  page = requests.get('https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start='+str(page)+'&ref_=adv_nxt')
  doc = BeautifulSoup(page.text,'html.parser')
  movie_data = doc.find_all('div',attrs={'class':'lister-item mode-advanced'})
  sleep(randint(2,8))
  
  for store in movie_data:
    name = store.h3.a.text
    movie_name.append(name)

    release_year = store.h3.find('span', class_='lister-item-year text-muted unbold').text.replace('(','').replace(')','')
    year.append(release_year)

    run_time = store.p.find('span', class_='runtime').text.replace('\tmin','')
    time.append(run_time)

    rate = store.find('div', class_='inline-block ratings-imdb-rating').text.replace('\n', '')
    rating.append(rate)

    if store.find('span', class_='metascore') == None: 
      meta = 'NaN'
    else:
      meta = store.find('span', class_='metascore').text.strip()
    metascore.append(meta)

    voting = store.find_all('span',{'name':'nv'})
    votes.append(voting[0].text)
    grosses = (voting[1].text) if len(voting) >1 else 'NaN'
    gross.append(grosses)

    if store.find('span',class_='certificate') == None: 
      certi = 'NaN'
    else:
      certi = store.find('span',class_='certificate').text.strip()
    certificate.append(certi)

    type_of_movie = store.find('span', class_='genre').text.strip()
    genre.append(type_of_movie)  

    desc = store.find_all('p',class_='text-muted')
    description.append(desc[1].text.strip())

    dir = store.find_all('a')
    director.append(dir[-6].text.strip())

    posterLink = store.find('img',class_='loadlate')
    poster.append(posterLink.attrs['loadlate'])

# Create CSV file(s) with the extracted information

In [5]:
imdb_dataset = pd.DataFrame({'Poster Link':poster, 'Title':movie_name, 'Release Year':year, 'Certificate':certificate,
                             'Runtime':time, 'Genre':genre,'Votes':votes, 'Rating':rating, 'Description':description, 'metascore':metascore,
                             'Director':director,'Gross':gross})

In [8]:
imdb_dataset

,Poster Link,Title,Release Year,Certificate,Runtime,Genre,Votes,Rating,Description,metascore,Director,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,R,142 min,Drama,"2,620,540",9.3,Two imprisoned men bond over a number of years...,81,Frank Darabont,$28.34M
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,R,175 min,"Crime, Drama","1,814,839",9.2,The aging patriarch of an organized crime dyna...,100,Francis Ford Coppola,$134.97M
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,PG-13,152 min,"Action, Crime, Drama","2,592,377",9.0,When the menace known as the Joker wreaks havo...,84,Christopher Nolan,$534.86M
3,https://m.media-amazon.com/images/M/MV5BNzA5ZD...,The Lord of the Rings: The Return of the King,2003,PG-13,201 min,"Action, Adventure, Drama","1,797,943",9.0,Gandalf and Aragorn lead the World of Men agai...,94,Peter Jackson,$377.85M
4,https://m.media-amazon.com/images/M/MV5BNDE4OT...,Schindler's List,1993,R,195 min,"Biography, Drama, History","1,330,949",9.0,"In German-occupied Poland during World War II,...",94,Steven Spielberg,$96.90M
...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://m.media-amazon.com/images/M/MV5BYmFlNT...,Sabrina,1954,Passed,113 min,"Comedy, Drama, Romance","64,787",7.6,A playboy becomes interested in the daughter o...,72,X,NaN
996,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,From Here to Eternity,1953,Passed,118 min,"Drama, Romance, War","46,961",7.6,"At a U.S. Army base in 1941 Hawaii, a private ...",85,X,$30.50M
997,https://m.media-amazon.com/images/M/MV5BMTQwMz...,Snow White and the Seven Dwarfs,1937,Approved,83 min,"Animation, Adventure, Family","197,319",7.6,Exiled into the dangerous forest by her wicked...,95,Perce Pearce,$184.93M
998,https://m.media-amazon.com/images/M/MV5BMTY5OD...,The 39 Steps,1935,Approved,86 min,"Crime, Mystery, Thriller","56,814",7.6,A man in London tries to help a counter-espion...,93,X,NaN


In [7]:
imdb_dataset.to_csv('imdb_dataset', index=None)